# Amazon SageMaker で Detectron2 と SKU-110K データセットを使って物体検出

このノートブックでは、Amazon SageMaker で Detectron2 の物体検出モデルを fine-tuning する方法をご紹介します。このノートブックについては、[こちらの AWS blog](https://aws.amazon.com/jp/blogs/news/object-detection-with-detectron2-on-amazon-sagemaker/) で解説しています。

ml.p3.8xlarge などハイスペックのインスタンスを使用するので、料金にご注意ください。インスタンスごとの料金は [こちらのサイト](https://aws.amazon.com/jp/sagemaker/pricing/) で確認できます。

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#背景" data-toc-modified-id="背景-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>背景</a></span></li><li><span><a href="#セットアップ" data-toc-modified-id="セットアップ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>セットアップ</a></span></li><li><span><a href="#データセットの準備" data-toc-modified-id="データセットの準備-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>データセットの準備</a></span><ul class="toc-item"><li><span><a href="#SKU-110K-データセットのダウンロード" data-toc-modified-id="SKU-110K-データセットのダウンロード-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>SKU-110K データセットのダウンロード</a></span></li><li><span><a href="#画像の前処理" data-toc-modified-id="画像の前処理-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>画像の前処理</a></span></li><li><span><a href="#アノテーションデータの前処理" data-toc-modified-id="アノテーションデータの前処理-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>アノテーションデータの前処理</a></span></li></ul></li><li><span><a href="#Amazon-SageMaker-を使って学習" data-toc-modified-id="Amazon-SageMaker-を使って学習-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Amazon SageMaker を使って学習</a></span><ul class="toc-item"><li><span><a href="#学習用コンテナのビルド" data-toc-modified-id="学習用コンテナのビルド-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>学習用コンテナのビルド</a></span></li><li><span><a href="#SageMaker-学習ジョブの設定" data-toc-modified-id="SageMaker-学習ジョブの設定-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>SageMaker 学習ジョブの設定</a></span></li></ul></li><li><span><a href="#Amazon-SageMaker-のハイパーパラメタチューニング" data-toc-modified-id="Amazon-SageMaker-のハイパーパラメタチューニング-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Amazon SageMaker のハイパーパラメタチューニング</a></span></li><li><span><a href="#Amazon-SageMaker-でのモデルデプロイ" data-toc-modified-id="Amazon-SageMaker-でのモデルデプロイ-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Amazon SageMaker でのモデルデプロイ</a></span></li><li><span><a href="#推論結果の可視化" data-toc-modified-id="推論結果の可視化-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>推論結果の可視化</a></span></li><li><span><a href="#リソースの削除" data-toc-modified-id="リソースの削除-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>リソースの削除</a></span></li></ul></div>

## 背景

Detectron2 は、物体検出アルゴリズムを実装したコンピュータビジョンフレームワークです。Facebook の AI リサーチチームによって開発されました。Detecton2 の祖先である Detectron は完全に Caffe で記述されていましたが、Detecton2 は PyTorch でリファクタリングされており、高速な実験とイテレーションを可能にします。Detectron2 は、物体検出、セマンティックセグメンテーション、ポーズ推定などの最先端のモデルを含む豊富な model zoo を備えています。モジュール式の設計により、Detetron2は容易に拡張可能であり、その結果、最先端の研究プロジェクトをその上に実装することができます。

このノートブックでは、Detectron2 を用いて  [SKU110k-dataset](https://github.com/eg4000/SKU110K_CVPR19) のモデルを学習・評価します。このオープンソースのデータセットには、小売店の棚の画像が含まれています。各画像には約150個のオブジェクトが含まれており、密集したシーンの物体検出アルゴリズムのテストに適しています。バウンディングボックスは、製品のカテゴリを区別することなく、SKU に関連付けられています。

このノートブックでは、Detectron2 の model zoo から物体検出モデルを使用しています。そして、Amazon SageMaker MLプラットフォームを利用して、SKU110kデータセットで事前に学習されたモデルを fine-tuning し、学習されたモデルを推論用にデプロイします。

**注意：このノートブックを Amazon SageMaker ノートブックインスタンスで使用する場合は、インスタンスにアタッチする EBS ボリュームサイズを 80GB くらいに設定するのがおすすめです。50GB 程度ですと、コンテナイメージをビルドする際に容量不足になることがあります。**

ノートブックインスタンスを使用している場合、コンテナイメージビルドの際の容量不足を回避するために以下のセルのコメントアウトを外してから実行して docker 関連のファイルの保存場所を変更してください。

In [ ]:
# %%bash

# sudo /etc/init.d/docker stop
# sudo mv /var/lib/docker /home/ec2-user/SageMaker/docker
# sudo ln -s /home/ec2-user/SageMaker/docker /var/lib/docker
# sudo /etc/init.d/docker start

## セットアップ

**注意：Sagemaker Notebook インスタンスまたは Sagemaker Studio インスタンスを使用してこのノートブックを実行する場合は、`AmazonSageMakerFullAccess` と `AmazonEC2ContainerRegistryFullAccess` ポリシーが付与された IAMロールを使用していることを確認してください。足りないポリシーがあれば、以下の手順で追加してください。**

1. [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/) を開く
1. **ノートブックインスタンス** を開いて現在使用しているノートブックインスタンスを選択する
1. **アクセス許可と暗号化** の部分に表示されている IAM ロールへのリンクをクリックする
1. IAM ロールの ARN は後で使用するのでメモ帳などにコピーしておく
1. **ポリシーをアタッチします** をクリックして `AmazonEC2ContainerRegistryFullAccess` を検索する
1. `AmazonEC2ContainerRegistryFullAccess` の横のチェックボックスをオンにする
1. 必要なポリシーの数だけ、同様の手順でポリシーを検索しチェックボックスをオンにして **ポリシーのアタッチ** をクリックする

まず、必要なPythonライブラリをインポートし、いくつかの共通パラメタを設定します。

In [ ]:
import boto3
import sagemaker

assert (
    sagemaker.__version__.split(".")[0] == "2"
), "Please upgrade SageMaker Python SDK to version 2"

In [ ]:
bucket = sagemaker.session.Session().default_bucket()
prefix_data = "detectron2/data"
prefix_model = "detectron2/training_artefacts"
prefix_code = "detectron2/model"
prefix_predictions = "detectron2/predictions"
local_folder = "cache"   # cache folder used to store downloaded data - not versioned


sm_session = sagemaker.Session(default_bucket=bucket)
role = sagemaker.get_execution_role()
region = sm_session.boto_region_name
account = sm_session.account_id()

# if bucket doesn't exist, create one
s3_resource = boto3.resource("s3")
if not s3_resource.Bucket(bucket) in s3_resource.buckets.all():
    s3_resource.create_bucket(
        Bucket=bucket, CreateBucketConfiguration={"LocationConstraint": region}
    )

## データセットの準備

SKU110K を学習用に用意するには、以下の作業が必要です。

- SKU-110K のデータセットをダウンロードし、解凍する。
- ファイル名の prefix に従って、画像を3つのチャンネル（学習、検証、テスト）に分割する。
- PIL.Image.load()で読み込めないような、破損した画像（および対応するアノテーション）を削除する。
- 画像チャンネルをS3バケットにアップロードする。
- アノテーションデータを拡張マニフェストファイル形式に変換し、これらのファイルをS3にアップロードする。


In [ ]:
import json
import os
import tarfile
import tempfile
from datetime import datetime
from pathlib import Path
from typing import Mapping, Optional, Sequence
from urllib import request

import boto3
import numpy as np
import pandas as pd
from tqdm import tqdm

### SKU-110K データセットのダウンロード

解凍したデータセットの合計サイズは 12.2GBです。これに合わせて、ノートブックインスタンスのボリュームサイズを設定してください。30GB程度のボリュームサイズをおすすめします。

⚠️ データセットのダウンロードと解凍には15〜20分程度かかります。

In [ ]:
! wget -P cache http://trax-geometry.s3.amazonaws.com/cvpr_challenge/SKU110K_fixed.tar.gz

In [ ]:
sku_dataset_dirname = "SKU110K_fixed"
assert Path(
    local_folder
).exists(), f"Set wget directory-prefix to {local_folder} in the previous cell"


def track_progress(members):
    i = 0
    for member in members:
        if i % 100 == 0:
            print(".", end="")
        i += 1
        yield member


if not (Path(local_folder) / sku_dataset_dirname).exists():
    compressed_file = tarfile.open(
        name=os.path.join(local_folder, sku_dataset_dirname + ".tar.gz")
    )
    compressed_file.extractall(
        path=local_folder, members=track_progress(compressed_file)
    )
else:
    print(f"Using the data in `{local_folder}` folder")

### 画像の前処理

In [ ]:
path_images = Path(local_folder) / sku_dataset_dirname / "images"
assert path_images.exists(), f"{path_images} not found"

prefix_to_channel = {
    "train": "training",
    "val": "validation",
    "test": "test",
}
for channel_name in prefix_to_channel.values():
    if not (path_images.parent / channel_name).exists():
        (path_images.parent / channel_name).mkdir()

for path_img in path_images.iterdir():
    for prefix in prefix_to_channel:
        if path_img.name.startswith(prefix):
            path_img.replace(
                path_images.parent / prefix_to_channel[prefix] / path_img.name
            )

Detectron2 は Pillow ライブラリを使って画像を読み込んでいます。SKU データセットに含まれる一部の画像が破損しており、データローダが IOError 例外を発生させることがわかりました。そこで、それらの画像をデータセットから削除します。

In [ ]:
CORRUPTED_IMAGES = {
    "training": ("train_4222.jpg", "train_5822.jpg", "train_882.jpg", "train_924.jpg"),
    "validation": tuple(),
    "test": ("test_274.jpg", "test_2924.jpg"),
}

In [ ]:
for channel_name in prefix_to_channel.values():
    for img_name in CORRUPTED_IMAGES[channel_name]:
        try:
            (path_images.parent / channel_name / img_name).unlink()
            print(f"{img_name} removed from channel {channel_name} ")
        except FileNotFoundError:
            print(f"{img_name} not in channel {channel_name}")

In [ ]:
for channel_name in prefix_to_channel.values():
    print(
        f"Number of {channel_name} images = {sum(1 for x in (path_images.parent / channel_name).glob('*.jpg'))}"
    )

データセットを S3 にアップロードします。 ⚠️ この処理には 10-15 分程度かかります。

In [ ]:
channel_to_s3_imgs = {}

for channel_name in prefix_to_channel.values():
    inputs = sm_session.upload_data(
        path=str(path_images.parent / channel_name),
        bucket=bucket,
        key_prefix=f"{prefix_data}/{channel_name}",
    )
    print(f"{channel_name} images uploaded to {inputs}")
    channel_to_s3_imgs[channel_name] = inputs

### アノテーションデータの前処理

SKU-110K データセットのアノテーションは csv ファイルで保存されています。ここでは、それらを [拡張マニフェストファイル](https://docs.aws.amazon.com/sagemaker/latest/dg/augmented-manifest.html) に変換しています。バウンディングボックスアノテーションの仕様については、[SageMakerのドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-data-output.html#sms-output-box) をご参照ください。


In [ ]:
def create_annotation_channel(
    channel_id: str,
    path_to_annotation: Path,
    bucket_name: str,
    data_prefix: str,
    img_annotation_to_ignore: Optional[Sequence[str]] = None,
) -> Sequence[Mapping]:
    r"""Change format from original to augmented manifest files

    Parameters
    ----------
    channel_id : str
        name of the channel, i.e. training, validation or test
    path_to_annotation : Path
        path to annotation file
    bucket_name : str
        bucket where the data are uploaded
    data_prefix : str
        bucket prefix
    img_annotation_to_ignore : Optional[Sequence[str]]
        annotation from these images are ignore because the corresponding images are corrupted, default to None

    Returns
    -------
    Sequence[Mapping]
        List of json lines, each lines contains the annotations for a single. This recreates the
        format of augmented manifest files that are generated by Amazon SageMaker GroundTruth
        labeling jobs
    """
    if channel_id not in ("training", "validation", "test"):
        raise ValueError(
            f"Channel identifier must be training, validation or test. The passed values is {channel_id}"
        )
    if not path_to_annotation.exists():
        raise FileNotFoundError(f"Annotation file {path_to_annotation} not found")

    df_annotation = pd.read_csv(
        path_to_annotation,
        header=0,
        names=(
            "image_name",
            "x1",
            "y1",
            "x2",
            "y2",
            "class",
            "image_width",
            "image_height",
        ),
    )

    df_annotation["left"] = df_annotation["x1"]
    df_annotation["top"] = df_annotation["y1"]
    df_annotation["width"] = df_annotation["x2"] - df_annotation["x1"]
    df_annotation["height"] = df_annotation["y2"] - df_annotation["y1"]
    df_annotation.drop(columns=["x1", "x2", "y1", "y2"], inplace=True)

    jsonlines = []
    for img_id in df_annotation["image_name"].unique():
        if img_annotation_to_ignore and img_id in img_annotation_to_ignore:
            print(
                f"Annotations for image {img_id} are neglected as the image is corrupted"
            )
            continue
        img_annotations = df_annotation.loc[df_annotation["image_name"] == img_id, :]
        annotations = []
        for (
            _,
            _,
            img_width,
            img_heigh,
            bbox_l,
            bbox_t,
            bbox_w,
            bbox_h,
        ) in img_annotations.itertuples(index=False):
            annotations.append(
                {
                    "class_id": 0,
                    "width": bbox_w,
                    "top": bbox_t,
                    "left": bbox_l,
                    "height": bbox_h,
                }
            )
        jsonline = {
            "sku": {
                "annotations": annotations,
                "image_size": [{"width": img_width, "depth": 3, "height": img_heigh,}],
            },
            "sku-metadata": {
                "job_name": f"labeling-job/sku-110k-{channel_id}",
                "class-map": {"0": "SKU"},
                "human-annotated": "yes",
                "objects": len(annotations) * [{"confidence": 0.0}],
                "type": "groundtruth/object-detection",
                "creation-date": datetime.now()
                .replace(second=0, microsecond=0)
                .isoformat(),
            },
            "source-ref": f"s3://{bucket_name}/{data_prefix}/{channel_id}/{img_id}",
        }
        jsonlines.append(jsonline)
    return jsonlines

In [ ]:
annotation_folder = Path(local_folder) / sku_dataset_dirname / "annotations"
channel_to_annotation_path = {
    "training": annotation_folder / "annotations_train.csv",
    "validation": annotation_folder / "annotations_val.csv",
    "test": annotation_folder / "annotations_test.csv",
}
channel_to_annotation = {}

for channel in channel_to_annotation_path:
    annotations = create_annotation_channel(
        channel,
        channel_to_annotation_path[channel],
        bucket,
        prefix_data,
        CORRUPTED_IMAGES[channel],
    )
    print(f"Number of {channel} annotations: {len(annotations)}")
    channel_to_annotation[channel] = annotations

In [ ]:
def upload_annotations(p_annotations, p_channel: str):
    rsc_bucket = boto3.resource("s3").Bucket(bucket)

    json_lines = [json.dumps(elem) for elem in p_annotations]
    to_write = "\n".join(json_lines)

    with tempfile.NamedTemporaryFile(mode="w") as fid:
        fid.write(to_write)
        rsc_bucket.upload_file(
            fid.name, f"{prefix_data}/annotations/{p_channel}.manifest"
        )

In [ ]:
for channel_id, annotations in channel_to_annotation.items():
    upload_annotations(annotations, channel_id)

学習セット、検証セット、テストセットに含まれる画像の数を確認し、アップロードや前処理に失敗した場合にユーザーが学習を開始する前に検出できるようにしましょう。

In [ ]:
channel_to_expected_size = {
    "training": 8215,
    "validation": 588,
    "test": 2934,
}

prefix_data = "detectron2/data"
bucket_rsr = boto3.resource("s3").Bucket(bucket)
for channel_name, exp_nb in channel_to_expected_size.items():
    nb_objs = len(
        list(bucket_rsr.objects.filter(Prefix=f"{prefix_data}/{channel_name}"))
    )
    assert (
        nb_objs == exp_nb
    ), f"The {channel_name} set should have {exp_nb} images but it contains {nb_objs} images"

## Amazon SageMaker を使って学習

SageMaker で学習ジョブを実行するには、以下の作業を行います。

- 学習コンテナを構築し、Amazon Elastic Container Registry (ECR) にプッシュする。コンテナにはすべてのランタイム依存ファイルと学習スクリプトが含まれる
- 学習クラスタの構成やモデルのハイパーパラメタを含む学習ジョブの構成を定義する
- 学習ジョブをスケジューリングし、その進捗を確認する


### 学習用コンテナのビルド
学習コンテナをビルドする前に、Pytorch のベースイメージを取得するための共有 ECR リポジトリと、プライベート ECR リポジトリで認証を行う必要があります。

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com

これからビルドするコンテナは、AWS が用意した Pytorch コンテナをベースイメージとして使用します。ベースイメージに Detecton2 の依存関係を追加し、学習スクリプトをコピーします。以下のセルを実行して Dockerfile の内容を確認します。

In [ ]:
%%bash

# execute this cell to review Docker container
pygmentize -l docker Dockerfile.sku110ktraining

次に、ローカルで Docker コンテナをビルドし、ECR リポジトリにプッシュすることで、SageMaker が学習時にこのコンテナをコンピュートノードにデプロイできるようにします。以下のコマンドを実行して、コンテナをビルドしてプッシュします。以下のセルの実行が完了するまで 5分以上かかることもあります。作成される Docker イメージのサイズは約5GBです。

In [ ]:
%%bash
./build_and_push.sh sagemaker-d2-train-sku110k latest Dockerfile.sku110ktraining

### SageMaker 学習ジョブの設定

設定項目としては以下があります。

- data configuration： train/test/valのデータセットをどこに保存するかを定義します。
- コンテナの設定
- モデルのハイパーパラメタの設定
- クラスタのサイズやインスタンスの種類、監視するメトリクスなどの学習ジョブのパラメタ

In [ ]:
import json

import boto3
from sagemaker.estimator import Estimator

In [ ]:
# Data configuration

training_channel = f"s3://{bucket}/{prefix_data}/training/"
validation_channel = f"s3://{bucket}/{prefix_data}/validation/"
test_channel = f"s3://{bucket}/{prefix_data}/test/"

annotation_channel = f"s3://{bucket}/{prefix_data}/annotations/"

classes = [
    "SKU",
]

In [ ]:
# Container configuration

container_name = "sagemaker-d2-train-sku110k"
container_version = "latest"
training_image_uri = (
    f"{account}.dkr.ecr.{region}.amazonaws.com/{container_name}:{container_version}"
)

In [ ]:
# Metrics to monitor during training, each metric is scraped from container Stdout

metrics = [
    {"Name": "training:loss", "Regex": "total_loss: ([0-9\\.]+)",},
    {"Name": "training:loss_cls", "Regex": "loss_cls: ([0-9\\.]+)",},
    {"Name": "training:loss_box_reg", "Regex": "loss_box_reg: ([0-9\\.]+)",},
    {"Name": "training:loss_rpn_cls", "Regex": "loss_rpn_cls: ([0-9\\.]+)",},
    {"Name": "training:loss_rpn_loc", "Regex": "loss_rpn_loc: ([0-9\\.]+)",},
    {"Name": "validation:loss", "Regex": "total_val_loss: ([0-9\\.]+)",},
    {"Name": "validation:loss_cls", "Regex": "val_loss_cls: ([0-9\\.]+)",},
    {"Name": "validation:loss_box_reg", "Regex": "val_loss_box_reg: ([0-9\\.]+)",},
    {"Name": "validation:loss_rpn_cls", "Regex": "val_loss_rpn_cls: ([0-9\\.]+)",},
    {"Name": "validation:loss_rpn_loc", "Regex": "val_loss_rpn_loc: ([0-9\\.]+)",},
]

In [ ]:
# Training instance type

training_instance = "ml.p3.8xlarge"
if training_instance.startswith("local"):
    training_session = sagemaker.LocalSession()
    training_session.config = {"local": {"local_code": True}}
else:
    training_session = sm_session

学習ジョブでは，以下のハイパーパラメタを使用しています．自由に変更して実験してみてください。

In [ ]:
# Model Hyperparameters

od_algorithm = "faster_rcnn"  # choose one in ("faster_rcnn", "retinanet")
training_job_hp = {
    # Dataset
    "classes": json.dumps(classes),
    "dataset-name": json.dumps("sku110k"),
    "label-name": json.dumps("sku"),
    # Algo specs
    "model-type": json.dumps(od_algorithm),
    "backbone": json.dumps("R_101_FPN"),
    # Data loader
    "num-iter": 900,
    "log-period": 500,
    "batch-size": 16,
    "num-workers": 8,
    # Optimization
    "lr": 0.005,
    "lr-schedule": 3,
    # Faster-RCNN specific
    "num-rpn": 517,
    "bbox-head-pos-fraction": 0.2,
    "bbox-rpn-pos-fraction": 0.4,
    # Prediction specific
    "nms-thr": 0.2,
    "pred-thr": 0.1,
    "det-per-img": 300,
    # Evaluation
    "evaluation-type": "fast",
}

SageMaker 学習ジョブでモデルを学習してみましょう。学習状況は SageMaker コンソールのメニューで「トレーニング」→「トレーニングジョブ」でジョブ一覧画面から確認可能です。学習ジョブが完了するまで 20分程度かかります。

In [ ]:
# Compile Sagemaker Training job object and start training

d2_estimator = Estimator(
    image_uri=training_image_uri,
    role=role,
    sagemaker_session=training_session,
    instance_count=2,
    instance_type=training_instance,
    hyperparameters=training_job_hp,
    metric_definitions=metrics,
    output_path=f"s3://{bucket}/{prefix_model}",
    base_job_name=f"detectron2-{od_algorithm.replace('_', '-')}",
)

d2_estimator.fit(
    {
        "training": training_channel,
        "validation": validation_channel,
        "test": test_channel,
        "annotation": annotation_channel,
    },
    wait=False,
)

## Amazon SageMaker のハイパーパラメタチューニング

SageMaker SDK には `tuner` モジュールが付属しており、これを使って最適なハイパーパラメタを探すことができます（詳細は [こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)）。ここでは、検証データの loss を最小化することを目的として、異なるハイパーパラメタを用いていくつかの実験を行ってみましょう。

最適化されるハイパーパラメタを定義する `hparams_range` は自由に変更してください。⚠️ 注意点として、チューニングジョブは複数の学習ジョブを実行します。そのため、チューニングジョブが必要とする計算リソースの量に注意してください。

In [ ]:
from sagemaker.tuner import (
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
    IntegerParameter,
)

od_algorithm = "retinanet"  # choose one in ("faster_rcnn", "retinanet")

In [ ]:
hparams_range = {
    "lr": ContinuousParameter(0.0005, 0.1),
}
if od_algorithm == "faster_rcnn":
    hparams_range.update(
        {
            "bbox-rpn-pos-fraction": ContinuousParameter(0.1, 0.5),
            "bbox-head-pos-fraction": ContinuousParameter(0.1, 0.5),
        }
    )
elif od_algorithm == "retinanet":
    hparams_range.update(
        {
            "focal-loss-gamma": ContinuousParameter(2.5, 5.0),
            "focal-loss-alpha": ContinuousParameter(0.3, 1.0),
        }
    )
else:
    assert False, f"{od_algorithm} not supported"

In [ ]:
obj_metric_name = "validation:loss"
obj_type = "Minimize"
metric_definitions = [
    {"Name": "training:loss", "Regex": "total_loss: ([0-9\\.]+)",},
    {"Name": "training:loss_cls", "Regex": "loss_cls: ([0-9\\.]+)",},
    {"Name": "training:loss_box_reg", "Regex": "loss_box_reg: ([0-9\\.]+)",},
    {"Name": obj_metric_name, "Regex": "total_val_loss: ([0-9\\.]+)",},
    {"Name": "validation:loss_cls", "Regex": "val_loss_cls: ([0-9\\.]+)",},
    {"Name": "validation:loss_box_reg", "Regex": "val_loss_box_reg: ([0-9\\.]+)",},
]

ハイパーパラメタチューニングのための `estimator` を作成します。`use_spot_instances`、`max_run`、`max_wait` のコメントアウトを外すとスポットインスタンスを使ったハイパーパラメタチューニングが可能です。ただし、利用可能なスポットインスタンスがない場合、指定された時間待機したのちジョブが終了しますのでご注意ください。

In [ ]:
fixed_hparams = {
    # Dataset
    "classes": json.dumps(classes),
    "dataset-name": json.dumps("sku110k"),
    "label-name": json.dumps("sku"),
    # Algo specs
    "model-type": json.dumps(od_algorithm),
    "backbone": json.dumps("R_101_FPN"),
    # Data loader
    "num-iter": 9000,
    "log-period": 500,
    "batch-size": 16,
    "num-workers": 8,
    # Optimization
    "lr-schedule": 3,
    # Prediction specific
    "nms-thr": 0.2,
    "pred-thr": 0.1,
    "det-per-img": 300,
    # Evaluation
    "evaluation-type": "fast",
}

hpo_estimator = Estimator(
    image_uri=training_image_uri,
    role=role,
    sagemaker_session=sm_session,
    instance_count=1,
    instance_type="ml.p3.8xlarge",
    hyperparameters=fixed_hparams,
    output_path=f"s3://{bucket}/{prefix_model}",
#     use_spot_instances=True,  # Use spot instances to spare a
#     max_run=2 * 60 * 60,
#     max_wait=3 * 60 * 60,
)

In [ ]:
tuner = HyperparameterTuner(
    hpo_estimator,
    obj_metric_name,
    hparams_range,
    metric_definitions,
    objective_type=obj_type,
    max_jobs=2,
    max_parallel_jobs=2,
    base_tuning_job_name=f"hpo-d2-{od_algorithm.replace('_', '-')}",
)

`fit` を実行することで、ハイパーパラメタチューニングが開始します。このノートブックの設定の場合、チューニングジョブが完了するまでに 2時間程度かかります。2つ上のセルの `fixed_hparams` の中の `num-iter` の値を変えることでチューニングジョブにかかる時間を調整できます。

In [ ]:
tuner.fit(
    inputs={
        "training": training_channel,
        "validation": validation_channel,
        "test": test_channel,
        "annotation": annotation_channel,
    },
    wait=False,
)

チューニングジョブが開始したら、以下のセルを実行して状況を確認します。チューニングジョブの開始に少し時間がかかるので、上記セルを実行してから 1分ほど経ってから以下のセルを実行してください。

In [ ]:
# Let's review outcomes of HyperParameter search

hpo_tuning_job_name = tuner.latest_tuning_job.name
bayes_metrics = sagemaker.HyperparameterTuningJobAnalytics(
    hpo_tuning_job_name
).dataframe()
bayes_metrics.sort_values(["FinalObjectiveValue"], ascending=True)

## Amazon SageMaker でのモデルデプロイ

チューニングジョブの完了を待つ間に、推論用コンテナイメージを作成します。モデルの学習と同様に、SageMaker は推論を実行するためにコンテナを使用します。

In [ ]:
%%bash

# execute this cell to review Docker container
pygmentize -l docker Dockerfile.sku110kserving

以下のセルを実行して、イメージ `Dockerfile.sku110kserving` で定義された Dockerコンテナを構築し、ECR にプッシュします。作成される Docker イメージのサイズは約5GBです。

In [ ]:
%%bash

./build_and_push.sh sagemaker-d2-serve latest Dockerfile.sku110kserving

バッチ推論、つまり大規模な画像の塊に対して推論を実行します。これには [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html) を使用します。

In [ ]:
from sagemaker.pytorch import PyTorchModel

**ここからは、HPO (Hyper Parameter Optimizer) ジョブが完了してから実行してください。**チューニングジョブをアタッチし、最適なモデルを取得します。

In [ ]:
from sagemaker.tuner import HyperparameterTuner

tuning_job_id = tuner.latest_tuning_job.name
attached_tuner = HyperparameterTuner.attach(tuning_job_id)

best_estimator = attached_tuner.best_estimator()

best_estimator.latest_training_job.describe()
training_job_artifact = best_estimator.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]

また、モデルアーティファクトの S3 URI を指定することもできます。このオプションを使いたい場合は、以下のコードのコメントを外してください。

In [ ]:
# training_job_artifact = best_estimator.model_data

In [ ]:
# Define parameters of inference container

serve_container_name = "sagemaker-d2-serve"
serve_container_version = "latest"
serve_image_uri = f"{account}.dkr.ecr.{region}.amazonaws.com/{serve_container_name}:{serve_container_version}"

inference_output = f"s3://{bucket}/{prefix_predictions}/{serve_container_name}/{Path(test_channel).name}_channel/{training_job_artifact.split('/')[-3]}"
inference_output

In [ ]:
# Compile SageMaker model object and configure Batch Transform job

model = PyTorchModel(
    name="d2-sku110k-model",
    model_data=training_job_artifact,
    role=role,
    sagemaker_session=sm_session,
    entry_point="predict_sku110k.py",
    source_dir="container_serving",
    image_uri=serve_image_uri,
    framework_version="1.6.0",
    code_location=f"s3://{bucket}/{prefix_code}",
)

transformer = model.transformer(
    instance_count=1,
    instance_type="ml.p3.2xlarge",  # "ml.p2.xlarge", #
    output_path=inference_output,
    max_payload=16,
)

以下のセルを実行して、バッチ変換ジョブを開始します。バッチ変換ジョブが完了するまでに 20分程度かかります。

In [ ]:
# Start Batch Transform job

transformer.transform(
    data=test_channel,
    data_type="S3Prefix",
    content_type="application/x-image",
    wait=False,
)

`TransformJobName` にバッチ変換ジョブ名を入れて以下のセルを実行すると、ジョブのステータスを知ることができます。

In [ ]:
sagemaker_client = boto3.client('sagemaker')
response = sagemaker_client.describe_transform_job(
    TransformJobName='<transform job name>'
)
response['TransformJobStatus']

## 推論結果の可視化

バッチ推論ジョブが完了したら、推論結果を可視化してみましょう。ここでは、テスト用画像からランダムに1枚選んで表示します。

In [ ]:
import io

import matplotlib
import matplotlib.patches as patches
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
def key_from_uri(s3_uri: str) -> str:
    """Get S3 object key from its URI"""
    return "/".join(Path(s3_uri).parts[2:])


bucket_rsr = boto3.resource("s3").Bucket(bucket)
predict_objs = list(
    bucket_rsr.objects.filter(Prefix=key_from_uri(inference_output) + "/")
)
img_objs = list(bucket_rsr.objects.filter(Prefix=key_from_uri(test_channel)))

In [ ]:
COLORS = [
    (0, 200, 0),
]


def plot_predictions_on_image(
    p_img: np.ndarray, p_preds: Mapping, score_thr: float = 0.5, show=True
) -> plt.Figure:
    r"""Plot bounding boxes predicted by an inference job on the corresponding image

    Parameters
    ----------
    p_img : np.ndarray
        input image used for prediction
    p_preds : Mapping
        dictionary with bounding boxes, predicted classes and confidence scores
    score_thr : float, optional
        show bounding boxes whose confidence score is bigger than `score_thr`, by default 0.5
    show : bool, optional
        show figure if True do not otherwise, by default True

    Returns
    -------
    plt.Figure
        figure handler

    Raises
    ------
    IOError
        If the prediction dictionary `p_preds` does not contain one of the required keys:
        `pred_classes`, `pred_boxes` and `scores`
    """
    for required_key in ("pred_classes", "pred_boxes", "scores"):
        if required_key not in p_preds:
            raise IOError(f"Missing required key: {required_key}")

    fig, fig_axis = plt.subplots(1)
    fig_axis.imshow(p_img)
    for class_id, bbox, score in zip(
        p_preds["pred_classes"], p_preds["pred_boxes"], p_preds["scores"]
    ):
        if score < score_thr:
            break  # bounding boxes are sorted by confidence score in descending order
        rect = patches.Rectangle(
            (bbox[0], bbox[1]),
            bbox[2] - bbox[0],
            bbox[3] - bbox[1],
            linewidth=1,
            edgecolor=[float(val) / 255 for val in COLORS[class_id]],
            facecolor="none",
        )
        fig_axis.add_patch(rect)
    plt.axis("off")
    if show:
        plt.show()
    return fig

以下のセルを実行すると、推論結果の画像が表示されます。ランダムで画像が表示されるので、何回か実行して結果を確認してみてください。

In [ ]:
matplotlib.rcParams["figure.dpi"] = 300

sample_id = np.random.randint(0, len(img_objs), 1)[0]

img_obj = img_objs[sample_id]
pred_obj = predict_objs[sample_id]

img = np.asarray(Image.open(io.BytesIO(img_obj.get()["Body"].read())))
preds = json.loads(pred_obj.get()["Body"].read().decode("utf-8"))

sample_fig = plot_predictions_on_image(img, preds, 0.40, True)

## リソースの削除

不要になったら、課金を停止するためにこのノートブックを実行したノートブックインスタンスを削除してください。なお、インスタンスを「停止」しただけでは EBS ボリュームへの課金は継続するので、完全に課金を止めるためにインスタンスを「停止」してから「削除」を実施してください。なお、削除したあとはインスタンスに保存されているファイルなどにアクセスすることはできません。